In [ ]:
import pandas as pd
import numpy as np
from haversine import haversine

import json
import urllib
import ssl

import folium

# Google Maps API, NAVER Maps API

In [ ]:
def crawling_g(initial, final):
        #Google Directions API
        origin          = initial
        destination     = final
        mode            = "transit"
        departure_time  = "now"
        key             = "-------Your Google API Key Here-------"

        url = "https://maps.googleapis.com/maps/api/directions/json?origin="+ origin \
                + "&destination=" + destination \
                + "&mode=" + mode \
                + "&departure_time=" + departure_time\
                + "&language=ko" \
                + "&key=" + key

        request         = urllib.request.Request(url)
        context         = ssl._create_unverified_context()
        response        = urllib.request.urlopen(request, context=context)
        responseText    = response.read().decode('utf-8')
        responseJson    = json.loads(responseText)

        return responseJson

In [ ]:
def crawling_n(start, goal) :
    #Naver Directions API
    client_id = "-------Your Naver API ID Here-------"
    client_secret = "-------Your Google API Key Here-------"
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start="+ start \
        + "&goal=" + goal
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if (res == 200) :
        response_body = response.read().decode('utf-8')
        responseJson = json.loads(response_body)
        return responseJson

In [ ]:
def parsing_g(directions):
    '''
    Inputs: the api result
    Outputs: total travel time and location of nearby station
    '''
    if directions['status'] == 'OK':
        # unit is seconds
        total_tt = directions['routes'][0]['legs'][0]['duration']['value']
        steps = directions['routes'][0]['legs'][0]['steps']
        mode_list = [i['travel_mode'] for i in steps]
        tt_list = [i['duration']['value'] for i in steps]
        if 'TRANSIT' in mode_list:
            s = mode_list.index('TRANSIT')
            station_loc = steps[s]['transit_details']['departure_stop']['location']
        else:
            station_loc = None
        return total_tt, station_loc, mode_list, tt_list
    else: # if api call is not valid, return None
        return None, None, None, None

In [ ]:
def parsing_n(directions):
    '''
    Inputs: the api result
    Outputs: total tollgates fare, taxi fare, fuel fare, times would be cost
    '''
    if directions['code'] == 0 :
        toll = directions['route']['traoptimal'][0]['summary']['tollFare']
        taxi = directions['route']['traoptimal'][0]['summary']['taxiFare']
        fuel = directions['route']['traoptimal'][0]['summary']['fuelPrice']
        time = directions['route']['traoptimal'][0]['summary']['duration']
        return toll, taxi, fuel, time
    else:
        return None, None, None, None

In [ ]:
def get_address(loc_str) :
    client_id = "-------Your Naver API ID Here-------"
    client_secret = "-------Your Google API Key Here-------"

    url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords="+loc_str+"&sourcecrs=epsg:4326&output=json&orders=addr,admcode"

    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if (res == 200) :
        response_body = response.read().decode('utf-8')
        responseJson = json.loads(response_body)
        return responseJson

# City Data

In [ ]:
city_loc = pd.read_csv("location_city.csv", encoding='cp949')
#Data Source -> S.Korean Public Data Portal

In [ ]:
city_loc.drop(17, inplace=True)

In [ ]:
city_loc

,시도,시도명,위도,경도,축척,행정구역최소X좌표,행정구역최소Y좌표,행정구역최대X좌표,행정구역최대Y좌표,매핑코드,상태,시도단축명,지진해일발생분류,CBS연계하위시군구갯수
0,11,서울특별시,37.594334,126.977656,25000.0,126.764264,37.428478,127.183535,37.701364,11000.0,Y,서울,NaN,25.0
1,26,부산광역시,35.105502,129.031892,5000.0,128.761407,34.932263,129.304070,35.388689,26000.0,Y,부산,Y,16.0
2,27,대구광역시,35.866572,128.593520,10000.0,128.350382,35.606778,128.763375,36.016459,27000.0,Y,대구,NaN,8.0
3,28,인천광역시,37.468389,126.482049,100000.0,124.592837,36.926850,126.793681,37.983802,28000.0,Y,인천,NaN,10.0
4,29,광주광역시,35.117018,126.949562,25000.0,126.646416,35.052092,127.022198,35.259303,29000.0,Y,광주,NaN,5.0
5,30,대전광역시,36.323839,127.475077,50000.0,127.246832,36.183266,127.557253,36.500203,30000.0,Y,대전,NaN,5.0
6,31,울산광역시,35.570971,129.309565,25000.0,128.970911,35.322050,129.464837,35.724718,31000.0,Y,울산,Y,5.0
7,36,세종특별자치시,36.560817,127.258755,100000.0,127.127658,36.406730,127.410806,36.733752,36000.0,Y,세종,NaN,1.0
8,41,경기도,37.313940,127.003597,25000.0,126.378668,36.893524,127.849448,38.292017,41000.0,Y,경기,NaN,31.0
9,42,강원도,37.890127,127.740070,100000.0,127.109253,37.028329,129.364009,38.616944,42000.0,Y,강원,Y,18.0


# Make Lattice on Map

In [ ]:
lattice_gap = {}

#Make Lattices on Map by 2km or 5km
#2km on Metropolitan City, 5km on Normal City as Gyongsangnamdo, Jeollanam-do, etc.

for i in range(len(city_loc)):
  if str(city_loc['시도명'].iloc[i])[-1:] == '시':
    lattice_gap[city_loc['시도단축명'].iloc[i]] = 0.02
  else:
    lattice_gap[city_loc['시도단축명'].iloc[i]] = 0.05

In [ ]:
local_lattice = {}

for i in range(len(city_loc)):
  local_lattice[city_loc['시도단축명'].iloc[i]] = [np.arange(float(city_loc['행정구역최소Y좌표'].iloc[i]), float(city_loc['행정구역최대Y좌표'].iloc[i]), lattice_gap[city_loc['시도단축명'].iloc[i]]), np.arange(float(city_loc['행정구역최소X좌표'].iloc[i]), float(city_loc['행정구역최대X좌표'].iloc[i]), lattice_gap[city_loc['시도단축명'].iloc[i]])]

In [ ]:
lattice_num = {}

for i in range(len(local_lattice.keys())):
  lattice_num[list(local_lattice.keys())[i]] = len(list(local_lattice.values())[i][0])*len(list(local_lattice.values())[i][1])

In [ ]:
lattice_num

{'서울': 294,
 '부산': 644,
 '대구': 441,
 '인천': 5883,
 '광주': 209,
 '대전': 256,
 '울산': 525,
 '세종': 255,
 '경기': 840,
 '강원': 1472,
 '충북': 700,
 '충남': 966,
 '전북': 702,
 '전남': 1710,
 '경북': 3280,
 '경남': 957,
 '제주': 323}

In [ ]:
get_address("126.8747,33.525369")['results'][0]['region']['area1']['name']

'제주특별자치도'

In [ ]:
real_lattice_location = {}

In [ ]:
templist = []

for i in local_lattice['서울'][0]:
  for j in local_lattice['서울'][1]:
    location_str = str(j)+','+str(i)
    try:
      if get_address(location_str)['results'][0]['region']['area1']['name'] == '서울특별시':
        templist.append((j, i))
    except:
      pass

real_lattice_location['서울'] = templist

In [ ]:
templist = []

for i in local_lattice['경기'][0]:
  for j in local_lattice['경기'][1]:
    location_str = str(j)+','+str(i)
    try:
      if get_address(location_str)['results'][0]['region']['area1']['name'] == '경기도':
        templist.append((j, i))
    except:
      pass

real_lattice_location['경기'] = templist

In [ ]:
templist = []

for i in local_lattice['부산'][0]:
  for j in local_lattice['부산'][1]:
    location_str = str(j)+','+str(i)
    try:
      if get_address(location_str)['results'][0]['region']['area1']['name'] == '부산광역시':
        templist.append((j, i))
    except:
      pass

real_lattice_location['부산'] = templist

In [ ]:
templist = []

for i in local_lattice['제주'][0]:
  for j in local_lattice['제주'][1]:
    location_str = str(j)+','+str(i)
    try:
      if get_address(location_str)['results'][0]['region']['area1']['name'] == '제주특별자치도':
        templist.append((j, i))
    except:
      pass

real_lattice_location['제주'] = templist

In [ ]:
print(len(real_lattice_location['서울']), len(real_lattice_location['부산']), len(real_lattice_location['경기']), len(real_lattice_location['제주']))

155 225 435 81


In [ ]:
with open('real_lattice_location.json', 'w') as f :
	json.dump(real_lattice_location, f)

# Directions API use

In [ ]:
with open('real_lattice_location.json', "r") as f:
    real_lattice_location = json.load(f)

In [ ]:
mobility_temp = []

for loc1 in real_lattice_location['서울']:
  templist = []
  for loc2 in real_lattice_location['서울']:
    if loc1 != loc2:
      templist.append({'Public Transport': crawling_g(str(loc1[1])+','+str(loc1[0]), str(loc2[1])+','+str(loc2[0])), 'Car/Taxi': crawling_n(str(loc1[1])+','+str(loc1[0]), str(loc2[1])+','+str(loc2[0]))})
    else:
      templist.append(np.nan)
  mobility_temp.append(templist)

seoul_mobility = pd.DataFrame(mobility_temp, columns=list(map(str, real_lattice_location['서울'])), index=list(map(str, real_lattice_location['서울'])))
seoul_mobility.to_json('Seoul_lattice_mobility.json')

In [ ]:
mobility_temp = []

for loc1 in real_lattice_location['부산']:
  templist = []
  for loc2 in real_lattice_location['부산']:
    if loc1 != loc2:
      templist.append({'Public Transport': crawling_g(str(loc1[1])+','+str(loc1[0]), str(loc2[1])+','+str(loc2[0])), 'Car/Taxi': crawling_n(str(loc1[1])+','+str(loc1[0]), str(loc2[1])+','+str(loc2[0]))})
    else:
      templist.append(np.nan)
  mobility_temp.append(templist)

busan_mobility = pd.DataFrame(mobility_temp, columns=list(map(str, real_lattice_location['부산'])), index=list(map(str, real_lattice_location['부산'])))
busan_mobility.to_json('Busan_lattice_mobility.json')

In [ ]:
mobility_temp = []

for loc1 in real_lattice_location['제주']:
  templist = []
  for loc2 in real_lattice_location['제주']:
    if loc1 != loc2:
      templist.append({'Public Transport': crawling_g(str(loc1[1])+','+str(loc1[0]), str(loc2[1])+','+str(loc2[0])), 'Car/Taxi': crawling_n(str(loc1[1])+','+str(loc1[0]), str(loc2[1])+','+str(loc2[0]))})
    else:
      templist.append(np.nan)
  mobility_temp.append(templist)

jeju_mobility = pd.DataFrame(mobility_temp, columns=list(map(str, real_lattice_location['제주'])), index=list(map(str, real_lattice_location['제주'])))
jeju_mobility.to_json('Jeju_lattice_mobility.json')

In [ ]:
mobility_temp = []

for loc1 in real_lattice_location['경기']:
  templist = []
  for loc2 in real_lattice_location['경기']:
    if loc1 != loc2:
      templist.append({'Public Transport': crawling_g(str(loc1[1])+','+str(loc1[0]), str(loc2[1])+','+str(loc2[0])), 'Car/Taxi': crawling_n(str(loc1[1])+','+str(loc1[0]), str(loc2[1])+','+str(loc2[0]))})
    else:
      templist.append(np.nan)
  mobility_temp.append(templist)

gyounggi_mobility = pd.DataFrame(mobility_temp, columns=list(map(str, real_lattice_location['경기'])), index=list(map(str, real_lattice_location['경기'])))
gyounggi_mobility.to_json('Gyounggi_lattice_mobility.json')